# Import

In [1]:
from fgbuster import component_model as c
from fgbuster.mixingmatrix import MixingMatrix
from pyoperators import MPI
import yaml
import numpy as np

from qubic.lib.Instrument.Qacquisition import QubicAcquisition
from qubic.lib.Instrument.Qinstrument import QubicInstrument
from qubic.lib.MapMaking.ComponentMapMaking.preset.preset import PresetInitialisation
from qubic.lib.MapMaking.ComponentMapMaking.Qcomponent_model import invModifiedBlackBody
from qubic.lib.MapMaking.NeuralNetworkMapMaking.operators.forward_ops import ForwardOps
from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Qsamplings import equ2gal, get_pointing
from qubic.lib.Qscene import QubicScene

comm = MPI.COMM_WORLD
%matplotlib inline

# QUBIC Instances

In [2]:
dictfilename = "qubic/qubic/dicts/pipeline_demo.dict"
d = qubicDict()
d.read_from_file(dictfilename)

center = equ2gal(d["RA_center"], d["DEC_center"])
d["nf_recon"] = 4
d["nf_sub"] = 4
d["MultiBand"] = True
d["nside"] = 128

d["synthbeam_kmax"] = 1
nf_sub = d["nf_sub"]
d["synthbeam_fraction"] = 1

d["use_synthbeam_fits_file"] = False
d["noiseless"] = True
d["photon_noise"] = False
d["npointings"] = 100
d["instrument_type"] = "UWB"
nf_recon = d["nf_recon"]

p = get_pointing(d)
s = QubicScene(d)
q = QubicInstrument(d)
acquisition = QubicAcquisition(q, p, s, d)

In [3]:
params_file = "/home/laclavere/Thesis/qubic/qubic/scripts/MapMaking/src/NNMM/params.yml"
pipeline = PresetInitialisation(comm).initialize(params_file)

with open(params_file, "r") as tf:
            params = yaml.safe_load(tf)

========= Initialization =========
    => Reading parameters file
========= Initialization =========
    => Checking simulation parameters
    => No error detected !
    => Getting job ID
    => Creating folders
========= External Data =========
    => Computing Planck frequency bands
========= QUBIC =========
    => Reading QUBIC dictionary
    => Building QUBIC operator
Info apcnb2325: Allocating (992000,9) elements = 136.23046875 MiB in FSRRotation3dMatrix.__init__.
Info apcnb2325: Allocating (992000,9) elements = 136.23046875 MiB in FSRRotation3dMatrix.__init__.
Info apcnb2325: Allocating (992000,9) elements = 136.23046875 MiB in FSRRotation3dMatrix.__init__.
Info apcnb2325: Allocating (992000,9) elements = 136.23046875 MiB in FSRRotation3dMatrix.__init__.
Info apcnb2325: Allocating (992000,9) elements = 136.23046875 MiB in FSRRotation3dMatrix.__init__.
Info apcnb2325: Allocating (992000,9) elements = 136.23046875 MiB in FSRRotation3dMatrix.__init__.
Info apcnb2325: Allocating (992

# CMM Mixing Matrix

In [4]:
nus = pipeline.mixingmatrix.nus_eff_in[:8]
beta = pipeline.mixingmatrix.beta_in
comps = pipeline.comp.components_model_in

In [5]:
print("nus", nus, "\n", nus.shape, "\n")
print("beta", beta, "\n", beta.shape, "\n")
print("comps", comps, "\n", type(comps), "\n", comps.__len__())

nus [135.50540829 144.29216391 153.6486907  163.61193508 198.74126549
 211.62850707 225.35141303 239.96417145] 
 (8,) 

beta [1.54] 
 (1,) 

comps [1, 136779.6159441515*(0.00666666666666667*nu)**(beta_d + 1)*expm1(0.01760867023799751*nu)**2*exp(-0.01760867023799751*nu)/(nu**2*(exp(0.00239962153668311*nu) - 1)*expm1(2.641300535699627)**2)] 
 <class 'list'> 
 2


In [6]:
pipeline.mixingmatrix.Amm_in

array([[1.        , 0.790388  ],
       [1.        , 0.91237467],
       [1.        , 1.05979777],
       [1.        , 1.23964247],
       [1.        , 2.12296121],
       [1.        , 2.57724473],
       [1.        , 3.16492336],
       [1.        , 3.93578095],
       [1.        , 0.60182162],
       [1.        , 0.65873138],
       [1.        , 0.71986554],
       [1.        , 0.78556457],
       [1.        , 0.8561977 ],
       [1.        , 0.93216516],
       [1.        , 1.01390064],
       [1.        , 1.10187394],
       [1.        , 1.19659392],
       [1.        , 1.29861159],
       [1.        , 1.62633099],
       [1.        , 1.83602336],
       [1.        , 2.07124257],
       [1.        , 2.33523549],
       [1.        , 2.63166526],
       [1.        , 2.96466503],
       [1.        , 3.33889863],
       [1.        , 3.75962916],
       [1.        , 4.23279655],
       [1.        , 4.76510525]])

# NNMM Mixing Matrix

In [7]:
forward_ops = ForwardOps(q, acquisition, s)
mm = forward_ops.op_mixing_matrix(nus, beta, comps)

In [8]:
mm

array([[1.        , 0.79038788],
       [1.        , 0.9123747 ],
       [1.        , 1.05979761],
       [1.        , 1.23964247],
       [1.        , 2.12296115],
       [1.        , 2.57724477],
       [1.        , 3.16492312],
       [1.        , 3.93578094]])

# Test inverse MM

In [9]:
inv_comps = []
# CMB
inv_comps += [c.CMB()]
# Dust
inv_comps += [invModifiedBlackBody(nu0=params["Foregrounds"]["CO"]["nu0"], temp=20)]

In [10]:
inv_mm = MixingMatrix(*inv_comps).eval(nus, *beta)

In [11]:
inv_mm

array([[1.        , 0.37519991],
       [1.        , 0.32503472],
       [1.        , 0.27982084],
       [1.        , 0.23922499],
       [1.        , 0.13968859],
       [1.        , 0.11506608],
       [1.        , 0.09370005],
       [1.        , 0.07534806]])

In [15]:
1 / 0.79038788

1.265201586846195